Validation of the classification based on the WFsim

Buggs in the WFsim that are important to keep in mind:
 -  There is no double photo-emission taken into account
 -  There may be only ~ 500 events before the WFsim crashes
 
 
Possible extensions:
 - Add afterpulse boolian to the 'truth' info
 - Do the same for the other detector types

In [1]:
import strax
import straxen

/home/joran/.conda/envs/strax3.6/lib/python3.6/site-packages/datashader-0.7.0-py3.6.egg/datashader/transfer_functions.py:21: FutureWarning: xarray subclass Image should explicitly define __slots__
  class Image(xr.DataArray):


In [2]:
import wfsim

In [3]:
import numpy as np
import pandas as pd

instruction_dtype = [('event_number', np.int), ('type', '<U2'), ('t', np.int), 
    ('x', np.float32), ('y', np.float32), ('z', np.float32), 
    ('amp', np.int), ('recoil', '<U2')]

def rand_instructions(c):
    n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
    c['total_time'] = c['chunk_size'] * c['nchunk']

    instructions = np.zeros(2 * n, dtype=instruction_dtype)
    uniform_times = c['total_time'] * (np.arange(n) + 0.5) / n
#     print(uniform_times)
    instructions['t'] = np.repeat(uniform_times, 2) * int(1e9)
    instructions['event_number'] = np.digitize(instructions['t'], 
         1e9 * np.arange(c['nchunk']) * c['chunk_size']) - 1
    instructions['type'] = np.tile(['s1', 's2'], n)
    instructions['recoil'] = ['er' for i in range(n * 2)]

    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    instructions['x'] = np.repeat(r * np.cos(t), 2)
    instructions['y'] = np.repeat(r * np.sin(t), 2)
    instructions['z'] = np.repeat(np.random.uniform(-100, 0, n), 2)

    nphotons = np.random.uniform(2000, 2050, n)
    nelectrons = 10 ** (np.random.uniform(1, 4, n))
    instructions['amp'] = np.vstack([nphotons, nelectrons]).T.flatten().astype(int)

    return instructions

c = dict(event_rate = 20, chunk_size=1, nchunk=20)
inst = rand_instructions(c)
_ = pd.DataFrame(inst).to_csv('test_uni.csv', index=False)

In [4]:
!cat test_uni.csv

event_number,type,t,x,y,z,amp,recoil
0,s1,25000000,17.44938,-11.746649,-3.123611,2031,er
0,s2,25000000,17.44938,-11.746649,-3.123611,107,er
0,s1,75000000,16.77672,45.265022,-52.73556,2025,er
0,s2,75000000,16.77672,45.265022,-52.73556,265,er
0,s1,125000000,-4.1272426,45.582954,-31.577175,2033,er
0,s2,125000000,-4.1272426,45.582954,-31.577175,18,er
0,s1,175000000,-15.931105,-16.553642,-48.383526,2038,er
0,s2,175000000,-15.931105,-16.553642,-48.383526,2398,er
0,s1,225000000,-22.541552,-7.0038066,-57.360905,2011,er
0,s2,225000000,-22.541552,-7.0038066,-57.360905,18,er
0,s1,275000000,-18.151354,30.176718,-42.190346,2033,er
0,s2,275000000,-18.151354,30.176718,-42.190346,64,er
0,s1,325000000,47.46436,4.9915886,-90.39314,2036,er
0,s2,325000000,47.46436,4.9915886,-90.39314,1881,er
0,s1,375000000,-24.108149,19.330503,-46.573277,2047,er
0,s2,375000000,-24.108149,19.330503,-46.573277,115,er
0,s1,425000000,-43.91111,-6.639574,-65.815834,2011,er
0,s2,425000000,-43.91111,-6.639574,-65.815834,959,er
0

In [5]:
st = strax.Context(
register=[wfsim.RawRecordsFromFax],
config=dict(fax_file='./test_uni.csv'),
**straxen.contexts.common_opts)

In [6]:
# Just some id from post-SR1, so the corrections work
run_id = '180519_1902'

In [7]:
# # Call for event_info so it immediately get processed as well
# !rm -r strax_data
# st.make(run_id, 'event_info')

In [8]:
# truth = st.get_df(run_id, 'truth')

# data = st.get_df(run_id, 'event_info')

In [9]:
# n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
# c['total_time'] = c['chunk_size'] * c['nchunk']
# timing_grid = np.linspace(0, c['total_time'], 10*n+1) * 1e9
# timing_grid

In [10]:
# truth

In [11]:
# truth['merge_index'] = np.digitize(truth['t'], timing_grid)
# data['merge_index'] = np.digitize(data['time'], timing_grid)

# truth.drop('event_number', axis=1, inplace=True)
# data = data.merge(truth[truth['type']=='s2'], how='outer', left_on='merge_index', right_on='merge_index')

In [12]:
# data

In [13]:
# import matplotlib.pyp/lot as plt
# from matplotlib.colors import LogNorm
# from multihist import Histdd, Hist1d

# plt.rcParams['figure.figsize'] = (10, 5)
# fig = plt.figure()

# ax = fig.add_subplot(121)
# mh = Histdd(data.s2_area, data.n_photon,
#             bins=[np.logspace(1, 6, 101), np.logspace(1, 6, 101)])
# plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
# plt.xscale('log'); plt.yscale('log')
# plt.xlabel('s2_area (rec)'); plt.ylabel('n_photons (truth)')
# ax = fig.add_subplot(122)
# mh = Histdd(data.n_photon, (data.s2_area-data.n_photon)/data.n_photon, 
#             bins=[np.logspace(1, 6, 101), np.linspace(-0.5, 0.5, 101)])
# plt.xlabel('s2_area (rec)'); plt.ylabel('s2_area (rec) / n_photons (truth) -1')
# plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
# plt.xscale('log')

# plt.show()

In [14]:
# truth = st.get_df(run_id, 'truth')
# data = st.get_df(run_id, 'event_info')

# truth['merge_index'] = np.digitize(truth['t'], timing_grid)
# data['merge_index'] = np.digitize(data['time'], timing_grid)

# truth.drop('event_number', axis=1, inplace=True)
# data = data.merge(truth[truth['type']=='s1'], how='outer', left_on='merge_index', right_on='merge_index')

In [15]:
# plt.rcParams['figure.figsize'] = (10, 5)
# fig = plt.figure()

# ax = fig.add_subplot(121)
# mh = Histdd(data.s1_area, data.n_photon,
#             bins=[np.logspace(2, 2.8, 101), np.logspace(2, 2.8, 101)])
# plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
# plt.xscale('log'); plt.yscale('log')
# plt.xlabel('s1_area (rec)'); plt.ylabel('n_photons (truth)')
# ax = fig.add_subplot(122)
# mh = Histdd(data.n_photon, (data.s1_area-data.n_photon)/data.n_photon, 
#             bins=[np.logspace(2, 2.8, 101), np.linspace(-0.5, 0.5, 101)])
# plt.pcolormesh(mh.bin_edges[0], mh.bin_edges[1], mh.histogram.T, norm=LogNorm())
# plt.xscale('log')

# plt.show()

In [ ]:
!rm -r strax_data
peaks = st.make(run_id, 'peak_basics')

Simulating Raw Records:   5%|▌         | 41/800 [00:05<01:51,  6.81it/s]

RawRecordsFromFax:140516042990816 waiting for chunk 1


Simulating Raw Records:  10%|█         | 81/800 [00:12<01:12,  9.94it/s]

RawRecordsFromFax:140516042990816 waiting for chunk 2


Simulating Raw Records:  15%|█▌        | 121/800 [00:20<02:24,  4.69it/s]

...

Simulating Raw Records:  85%|████████▌ | 681/800 [02:10<00:15,  7.59it/s]

RawRecordsFromFax:140516042990816 waiting for chunk 17


Simulating Raw Records:  90%|█████████ | 721/800 [02:19<00:17,  4.57it/s]

RawRecordsFromFax:140516042990816 waiting for chunk 18


Simulating Raw Records:  95%|█████████▌| 761/800 [02:27<00:07,  5.06it/s]

In [ ]:
peaks

In [ ]:
truth = st.get_df(run_id, 'truth')



In [ ]:
data = st.get_df(run_id, ['peak_basics','peak_classification'])

In [ ]:
truth['merge_index'] = np.digitize(truth['t'], timing_grid)
data['merge_index'] = np.digitize(data['time'], timing_grid)

In [ ]:
truth['merge_index']

In [ ]:
data['merge_index']

In [ ]:
data.dtypes

In [ ]:
truth['time'] = truth['t']
truth['endtime'] = truth['t_last_photon']
mask = truth['endtime'] < truth['t_last_electron']
truth['endtime'][mask] = truth['t_last_electron'][mask]

In [ ]:
plt.hist(truth['endtime']-truth['time'] )
plt.yscale('log')

In [ ]:
# %load match_peaks.py
"""Utility to match peaks from results of different processor versions / processor and simulator
Jelle Aalbers, Nikhef, September 2015

Updated for strax, September 2019
Mike Clark
Joran Angrvaare
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from tqdm import tqdm

import recarray_tools as rt
from multihist import Hist1d
try:
    from pax.datastructure import INT_NAN
except ImportError:
    INT_NAN = -99999


def match_peaks(allpeaks1, allpeaks2, matching_fuzz=1, unknown_types=(b'unknown', b'lone_hit', b'coincidence'),
                keep_tpc_only=True):
    """Perform peak matching between two numpy record arrays with fields: Event, left, right, type, area
    If a peak is split into many fragments (e.g. two close peaks split into three peaks),
    the results are unreliable and depend on which peak set is peaks1 and which is peaks2.

    Returns (allpeaks1, allpeaks2), each with three extra fields: id, outcome, matched_to
        id: unique number for each peak
        outcome: Can be one of:
            found:  Peak was matched 1-1 between peaks1 and peaks2 (type agrees, no other peaks in range).
                    Note that area, widths, etc. can still be quite different!
            missed: Peak is not present in the other list
            misid_as_XX: Peak is present in the other list, but has type XX
            merged: Peak is merged with another peak in the other list, the new 'super-peak' has the same type
            merged_to_XX: As above, but 'super-peak' has type XX
            split: Peak is split in the other list, but more than one fragment has the same type as the parent.
            chopped: As split, but one or several fragments are unclassified, exactly one has the correct type.
            split_and_unclassified: As split, but all fragments are unclassified in the other list.
            split_and_misid: As split, but at least one fragment has a different peak type.
        matched_to: id of matching in peak in the other list if outcome is found or misid_as_XX, INT_NAN otherwise.
    """

#     # Keep only tpc_peaks
#     if keep_tpc_only:
#         if 'detector' in allpeaks1.dtype.names:
#             allpeaks1 = allpeaks1[allpeaks1['detector'] == b'tpc']
#         if 'detector' in allpeaks2.dtype.names:
#             allpeaks2 = allpeaks2[allpeaks2['detector'] == b'tpc']

#     # Remove true photoionization afterpulse peaks (they were not in initial instruction file)
#     allpeaks1 = allpeaks1[allpeaks1['type'] != b'photoionization_afterpulse']
#     allpeaks2 = allpeaks2[allpeaks2['type'] != b'photoionization_afterpulse']

    # Append id, outcome and matched_to fields
    print("\tAppending extra fields...\n")
    allpeaks1 = rt.append_fields(allpeaks1,
                                 ('id', 'outcome', 'matched_to'),
                                 (np.arange(len(allpeaks1)),
                                  np.array(['missed'] * len(allpeaks1), dtype='S32'),
                                  INT_NAN * np.ones(len(allpeaks1), dtype=np.int64)))
    allpeaks2 = rt.append_fields(allpeaks2,
                                 ('id', 'outcome', 'matched_to'),
                                 (np.arange(len(allpeaks2)),
                                 np.array(['missed'] * len(allpeaks2), dtype='S32'),
                                 INT_NAN * np.ones(len(allpeaks2), dtype=np.int64)))

    # Group each peak by event in OrderedDict
    print("\tGrouping peaks 1 by event...\n")
    peaks1_by_event = rt.dict_group_by(allpeaks1, 'merge_index')
    print("\tGrouping peaks 2 by event...\n")
    peaks2_by_event = rt.dict_group_by(allpeaks2, 'merge_index')

    for event, peaks_1 in tqdm(peaks1_by_event.items(), desc='Matching peaks'):
        if event not in peaks2_by_event:
            continue
        peaks_2 = peaks2_by_event[event]

        for p1_i, p1 in enumerate(peaks_1):
            # Select all found peaks that overlap at least partially with the true peak
            selection = (peaks_2['time'] <= p1['endtime'] + matching_fuzz) & \
                        (peaks_2['endtime'] >= p1['time'] - matching_fuzz)
            matching_peaks = peaks_2[selection]

            if len(matching_peaks) == 0:
                # Peak was missed; that's the default outcome, no need to set anything
                pass

            elif len(matching_peaks) == 1:
                # A unique match! Hurray!
                p2 = matching_peaks[0]
                p1['matched_to'] = p2['id']
                p2['matched_to'] = p1['id']
                # Do the types match?
                if p1['type'] == p2['type']:
                    p1['outcome'] = p2['outcome'] = 'found'
                else:
                    if p1['type'] in unknown_types:
                        p2['outcome'] = 'unclassified'
                    else:
                        p2['outcome'] = 'misid_as_%s' % p1['type'].decode()
                    if p2['type'] in unknown_types:
                        p1['outcome'] = 'unclassified'
                    else:
                        p1['outcome'] = 'misid_as_%s' % p2['type'].decode()
                    # If the peaks are unknown in both sets, they will count as 'found'.
                    # Hmm....
                matching_peaks[0] = p2
            else:
                # More than one peak overlaps p1
                handle_peak_merge(parent=p1, fragments=matching_peaks, unknown_types=unknown_types)
                
            # matching_peaks is a copy, not a view, so we have to copy the results over to peaks_2 manually
            # Sometimes I wish python had references...
            for i_in_matching_peaks, i_in_peaks_2 in enumerate(np.where(selection)[0]):
                peaks_2[i_in_peaks_2] = matching_peaks[i_in_matching_peaks]

        # Match in reverse to detect merged peaks
        # >1 peaks in 1 may claim to be matched to a peak in 2, in which case we should correct the outcome...
        for p2_i, p2 in enumerate(peaks_2):
            selection = peaks_1['matched_to'] == p2['id']
            matching_peaks = peaks_1[selection]
            if len(matching_peaks) > 1:
                handle_peak_merge(parent=p2, fragments=matching_peaks, unknown_types=unknown_types)
                
            # matching_peaks is a copy, not a view, so we have to copy the results over to peaks_1 manually
            # Sometimes I wish python had references...
            for i_in_matching_peaks, i_in_peaks_1 in enumerate(np.where(selection)[0]):
                peaks_1[i_in_peaks_1] = matching_peaks[i_in_matching_peaks]
                    


    # Concatenate peaks again into result list
    # Necessary because group_by (and np.split inside that) returns copies, not views
    return np.concatenate(list(peaks1_by_event.values())), \
           np.concatenate(list(peaks2_by_event.values()))


def handle_peak_merge(parent, fragments, unknown_types):
    found_types = fragments['type']
    is_ok = found_types == parent['type']
    is_unknown = np.in1d(found_types, unknown_types)
    is_misclass = (True ^ is_ok) & (True ^ is_unknown)
    # We have to loop over the fragments to avoid making a copy
    for i in range(len(fragments)):
        if is_unknown[i] or is_misclass[i]:
            if parent['type'] in unknown_types:
                fragments[i]['outcome'] = 'merged_to_unknown'
            else:
                fragments[i]['outcome'] = 'merged_to_%s' % parent['type'].decode()
        else:
            fragments[i]['outcome'] = 'merged'
        # Link the fragments to the parent
        fragments[i]['matched_to'] = parent['id']
    if np.any(is_misclass):
        parent['outcome'] = 'split_and_misid'
    # All fragments are either ok or unknown
    # If more than one fragment is given the same class
    # as the parent peak, then call it "split"
    elif len(np.where(is_ok)[0]) > 1:
        parent['outcome'] = 'split'
    elif np.all(is_unknown):
        parent['outcome'] = 'split_and_unclassified'
    # If exactly one fragment out of > 1 fragments
    # is correctly classified, then call the parent chopped
    else:
        parent['outcome'] = 'chopped'
    # We can't link the parent to all fragments... link to the largest one:
    parent['matched_to'] = fragments[np.argmax(fragments['area'])]['id']


outcome_colors = {
    'found':            'darkblue',
    'chopped':          'mediumslateblue',

    'missed':           'red',
    'merged':           'turquoise',
    'split':            'purple',

    'misid_as_s2':      'orange',
    'misid_as_s1':      'goldenrod',
    'split_and_misid':  'darkorange',
    'merged_to_s2':     'chocolate',
    'merged_to_s1':     'sandybrown',
    'merged_to_unknown': 'khaki',

    'unclassified':     'green',
    'split_and_unclassified':     'seagreen',
    'merged_and_unclassified':    'limegreen',
}



def peak_matching_histogram(results, histogram_key, bins=10):
    """Make 1D histogram of peak matching results (=peaks with extra fields added by matagainst histogram_key"""

    if histogram_key not in results.dtype.names:
        raise ValueError('Histogram key %s should be one of the columns in results: %s' % (histogram_key,
                                                                                           results.dtype.names))

    # How many true peaks do we have in each bin in total?
    n_peaks_hist = Hist1d(results[histogram_key], bins)
    hists = {'_total': n_peaks_hist}

    for outcome in np.unique(results['outcome']):
        # Histogram the # of peaks that have this outcome
        hist = Hist1d(results[results['outcome'] == outcome][histogram_key],
                      bins=n_peaks_hist.bin_edges)
        outcome = outcome.decode()
        hists[outcome] = hist

    return hists


def plot_peak_matching_histogram(*args, **kwargs):
    hists = peak_matching_histogram(*args, **kwargs)
    _plot_peak_matching_histogram(hists)


def _plot_peak_matching_histogram(hists):
    """Make 1D histogram of peak matching results (=peaks with extra fields added by matagainst histogram_key"""

    n_peaks_hist = hists['_total']

    for outcome, hist in hists.items():
        hist = hist.histogram.astype(np.float)

        if outcome == '_total':
            continue

        print("\t%0.2f%% %s" % (100 * hist.sum()/n_peaks_hist.n, outcome))

        # Compute Errors on estimate of a proportion
        # Should have vectorized this... lazy
        # Man this code is ugly!!!!
        limits_d = []
        limits_u = []
        for i, x in enumerate(hist):
            limit_d, limit_u = binom_interval(x, total=n_peaks_hist.histogram[i])
            limits_d.append(limit_d)
            limits_u.append(limit_u)
        limits_d = np.array(limits_d)
        limits_u = np.array(limits_u)

        # Convert hist to proportion
        hist /= n_peaks_hist.histogram.astype('float')

        color = outcome_colors.get(outcome, np.random.rand(3,))
        plt.errorbar(x=n_peaks_hist.bin_centers,
                     y=hist,
                     yerr=[hist - limits_d, limits_u - hist],
                     label=outcome,
                     color=color,
                     linestyle='-' if outcome == 'found' else '',
                     marker='s')

        # Wald intervals: not so good
        # errors = np.sqrt(
        #     hist*(1-hist)/all_true_peaks_histogram
        # )
        # plt.errorbar(x=bin_centers, y=hist, yerr = errors, label=outcome)

    plt.xlim(n_peaks_hist.bin_edges[0], n_peaks_hist.bin_edges[-1])
    plt.ylabel('Fraction of peaks')
    plt.ylim(0, 1)
    plt.legend(loc='lower right', shadow=True)
    legend = plt.legend(loc='best', prop={'size': 10})
    if legend and legend.get_frame():
        legend.get_frame().set_alpha(0.8)


def binom_interval(success, total, conf_level=0.95):
    """Confidence interval on binomial - using Jeffreys interval
    Code stolen from https://gist.github.com/paulgb/6627336
    Agrees with http://statpages.info/confint.html for binom_interval(1, 10)
    """
    # TODO: special case for success = 0 or = total? see wikipedia
    quantile = (1 - conf_level) / 2.
    lower = beta.ppf(quantile, success, total - success + 1)
    upper = beta.ppf(1 - quantile, success + 1, total - success)
    # If something went wrong with a limit calculation, report the trivial limit
    if np.isnan(lower):
        lower = 0
    if np.isnan(upper):
        upper = 1
    return lower, upper


In [ ]:
data.values

In [ ]:

match_peaks(data, truth)

In [ ]:
np.in1d('time', data.keys().values) 